<a href="https://colab.research.google.com/github/omarafifi01/ARVR/blob/main/runapi%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive; drive.mount('/content/drive')


WORK = "/content/drive/MyDrive/telco_churn_api"
NB    = "/content/drive/MyDrive/Untitled18.ipynb"

import os, textwrap, json, time, sys, traceback, requests
os.chdir(WORK)
print("PWD:", os.getcwd())
!ls -lh


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
PWD: /content/drive/MyDrive/telco_churn_api
total 6.7M
-rw------- 1 root root 8.0K Aug 17 16:23 app.py
-rw------- 1 root root 2.2K Aug 16 14:02 feature_schema.json
-rw------- 1 root root 1.7M Aug 16 14:02 lgbm_model.pkl
-rw------- 1 root root 5.0M Aug 16 14:02 pipeline.pkl
drwx------ 2 root root 4.0K Aug 17 16:25 __pycache__
-rw------- 1 root root  13K Aug 16 14:02 requirements.txt
-rw------- 1 root root  195 Aug 17 16:25 server.log
-rw------- 1 root root   34 Aug 16 14:02 threshold.json


In [8]:
import traceback, importlib
try:
    import app
    importlib.reload(app)
    print("✅ app.py imports OK")
except Exception:
    print("❌ Import failed. Traceback below:")
    traceback.print_exc()


✅ app.py imports OK


In [9]:
!pkill -f "uvicorn" || true
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 --log-level debug > server.log 2>&1 &
!sleep 2
!tail -n 120 server.log


^C
                                                                                                                                                                                                                           INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1555]


In [11]:
import requests, pprint, time
BASE = "http://127.0.0.1:8000"

def safe_get(url):
    try:
        r = requests.get(url, timeout=5)
        print(url, "->", r.status_code, r.text[:120])
        return r
    except Exception as e:
        print(url, "-> ERROR:", repr(e))

print("health:")
safe_get(BASE + "/healthz")

print("\nschema (first few names):")
r = safe_get(BASE + "/schema")
schema = r.json() if r and r.ok else []

def find_key(part):
    part = part.lower()
    for s in schema:
        if part in s["name"].lower():
            return s["name"]
    return None

k_tenure   = find_key("tenure")
k_monthly  = find_key("monthly")   # e.g., MonthlyCharges
k_contract = find_key("contract")
k_internet = find_key("internet")
k_payment  = find_key("payment")

example = {s["name"]: (0 if s["type"]=="number" else (s.get("allowed",[None])[0] or None)) for s in schema}
if k_tenure:   example[k_tenure]   = 2
if k_monthly:  example[k_monthly]  = 95
if k_contract: example[k_contract] = "Month-to-month"
if k_internet: example[k_internet] = "Fiber optic"
if k_payment:  example[k_payment]  = "Electronic check"

print("\n/predict:")
try:
    pr = requests.post(BASE + "/predict", json=example, timeout=10)
    print("status:", pr.status_code); pprint.pp(pr.json())
except Exception as e:
    print("predict error:", repr(e))

print("\n/chat:")
msg = "Month-to-month plan, fiber optic internet, electronic check, ~$95 monthly, tenure 5 months."
try:
    cr = requests.post(BASE + "/chat", json={"text": msg}, timeout=20)
    print("status:", cr.status_code); pprint.pp(cr.json())
except Exception as e:
    print("chat error:", repr(e))


health:
http://127.0.0.1:8000/healthz -> 200 {"ok":true}

schema (first few names):
http://127.0.0.1:8000/schema -> 200 [{"name":"tenure","type":"number"},{"name":"MonthlyCharges","type":"number"},{"name":"TotalCharges","type":"number"},{"n

/predict:
status: 200
{'probability': 0.7592077479553313,
 'label': 'Churn',
 'decision_threshold': 0.25485902895105866,
 'top_factors': [{'feature': 'TotalCharges', 'impact': 1.3105576566434203},
                 {'feature': 'Contract_Month-to-month',
                  'impact': 1.3034128664391895},
                 {'feature': 'tenure', 'impact': 1.1953321912410255},
                 {'feature': 'MonthlyCharges', 'impact': 0.4252197230705853},
                 {'feature': 'Paperless_Billing_No',
                  'impact': -0.28350285585770135}]}

/chat:
status: 200
{'answer': 'Estimated churn risk: 65%. Threshold: 25%.',
 'why_top_factors': 'tenure (+1.22); Contract_Month-to-month (+1.02); '
                    'MonthlyCharges (+0.30); Contract_

In [12]:
!pip -q install gradio==4.44.0 requests

import gradio as gr
import requests, json

BASE = "http://127.0.0.1:8000"

# ——— helpers ———
def get_schema():
    try:
        r = requests.get(f"{BASE}/schema", timeout=5)
        r.raise_for_status()
        return r.json()
    except Exception as e:
        return []

SCHEMA = get_schema()

def find_key(part):
    p = part.lower()
    for s in SCHEMA:
        if p in s["name"].lower():
            return s["name"]
    return None

def allowed(name):
    for s in SCHEMA:
        if s["name"] == name:
            return s.get("allowed", [])
    return []

K_TENURE   = find_key("tenure")
K_MONTHLY  = find_key("monthly")
K_CONTRACT = find_key("contract")
K_INTERNET = find_key("internet")
K_PAYMENT  = find_key("payment")

CONTRACT_CHOICES = allowed(K_CONTRACT) or ["Month-to-month","One year","Two year"]
INTERNET_CHOICES = allowed(K_INTERNET) or ["Fiber optic","DSL","No"]
PAYMENT_CHOICES  = allowed(K_PAYMENT)  or ["Electronic check","Mailed check","Bank transfer (automatic)","Credit card (automatic)"]

def payload_base():
    """Start with empty payload; fill what user provides."""
    return {}

# ——— backend calls ———
def call_predict(tenure, monthly, contract, internet, payment, extra_json):
    try:
        payload = payload_base()
        if K_TENURE and tenure is not None: payload[K_TENURE] = float(tenure)
        if K_MONTHLY and monthly is not None: payload[K_MONTHLY] = float(monthly)
        if K_CONTRACT and contract: payload[K_CONTRACT] = contract
        if K_INTERNET and internet: payload[K_INTERNET] = internet
        if K_PAYMENT and payment: payload[K_PAYMENT] = payment

        if extra_json.strip():
            try:
                extra = json.loads(extra_json)
                if isinstance(extra, dict):
                    payload.update(extra)
            except Exception as e:
                return {"error": f"extra_json parse error: {e}"}

        r = requests.post(f"{BASE}/predict", json=payload, timeout=15)
        return r.json()
    except Exception as e:
        return {"error": repr(e)}

def call_chat(message):
    try:
        r = requests.post(f"{BASE}/chat", json={"text": message}, timeout=30)
        return r.json()
    except Exception as e:
        return {"error": repr(e)}

# ——— UI ———
with gr.Blocks(theme="gradio/soft") as demo:
    gr.Markdown("# 📉 Telco Churn — Demo UI (Gradio)")
    with gr.Row():
        health_btn = gr.Button("Ping API /healthz")
        health_out = gr.Textbox(label="Health", interactive=False)
    def ping():
        try:
            return requests.get(f"{BASE}/healthz", timeout=5).text
        except Exception as e:
            return f"ERROR: {e}"
    health_btn.click(ping, outputs=health_out)

    with gr.Tab("💬 Chat (natural language)"):
        chat_in = gr.Textbox(label="Describe the customer", placeholder="e.g., Month-to-month, fiber optic, electronic check, ~$95 monthly, tenure 2 months.")
        chat_btn = gr.Button("Get Risk")
        chat_out = gr.JSON(label="Response")
        chat_btn.click(call_chat, inputs=chat_in, outputs=chat_out)

    with gr.Tab("🧮 Predict (structured)"):
        with gr.Row():
            tenure   = gr.Number(label=K_TENURE or "tenure", value=2, precision=0)
            monthly  = gr.Number(label=K_MONTHLY or "MonthlyCharges", value=95)
        with gr.Row():
            contract = gr.Dropdown(CONTRACT_CHOICES, value=CONTRACT_CHOICES[0], label=K_CONTRACT or "Contract")
            internet = gr.Dropdown(INTERNET_CHOICES, value=INTERNET_CHOICES[0], label=K_INTERNET or "Internet_Service")
            payment  = gr.Dropdown(PAYMENT_CHOICES, value=PAYMENT_CHOICES[0], label=K_PAYMENT or "Payment_Method")
        extra_json = gr.Textbox(label="(Optional) Extra JSON fields to include", placeholder='{"SeniorCitizen": 0, "Paperless_Billing": "Yes"}')
        pred_btn = gr.Button("Predict")
        pred_out = gr.JSON(label="Prediction")
        pred_btn.click(call_predict, inputs=[tenure, monthly, contract, internet, payment, extra_json], outputs=pred_out)

demo.launch(share=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.9/130.9 kB 9.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.29.0 requires websockets<15.1.0,>=13.0.0, but you have websockets 12.0 which is incompatible.
yfinance 0.2.65 requires websockets>=13.0, but you have websockets 12.0 which is incompatible.
dataproc-spark-connect 0.8.3 requires websockets>=14.0, but you have websockets 12.0 which is incompatible.


theme_schema%400.0.3.json: 0.00B [00:00, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()


/usr/local/lib/python3.11/dist-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Running on public URL: https://badce02800e1e6851d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
